<a href="https://colab.research.google.com/github/yesh9836/yesh9836-streamlitapps-/blob/main/MaheshProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/Mental disorder symptoms.csv")  #
desired_order = [
   "ag+1:629e","feeling.nervous", "trouble.in.concentration", "having.trouble.in.sleeping",
    "social.media.addiction", "having.nightmares", "change.in.eating",
    "feeling.tired", "sweating", "breathing.rapidly", "anger",
    'close.friend', "introvert", "feeling.negative", "avoids.people.or.activities",
     "blamming.yourself", "hallucinations", "panic",
    "hopelessness", "suicidal.thought", "popping.up.stressful.memory","Disorder"
]

# Reordering the columns
df = df[desired_order]
y=df["Disorder"]
x=df.drop("Disorder",axis=1)

In [ ]:
# preprocessing, converting diseases into numerical data
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
encoder.fit(y)
# Get the mapping as a DataFrame
label_mapping = pd.DataFrame({
    'label': encoder.classes_,
    'encoded': encoder.transform(encoder.classes_)
})

# Save to CSV
label_mapping.to_csv('label_encoder_mappings.csv', index=False)

X_train, X_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.3, random_state=42)
print(X_train.shape,y_train.shape)

(445, 21) (445,)


In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Initialize classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
dt = DecisionTreeClassifier(random_state=42)
svm = SVC(kernel='linear', probability=True, random_state=42)  # Set probability=True for soft voting
nb = GaussianNB()
lr = LogisticRegression(random_state=42)

# Create the VotingClassifier (with hard voting)
voting_clf = VotingClassifier(estimators=[
    ('rf', rf),
    ('dt', dt),
    ('svm', svm),
    ('nb', nb),
    ('lr', lr)
], voting='hard')  # For hard voting

# 2. Train the VotingClassifier
model=voting_clf.fit(X_train, y_train)
joblib.dump(model, 'model.pkl')

# 3. Predict on the test set
y_pred = model.predict(X_test)
# 4. Evaluate the model
# Reverse the encoding for predicted disease names
predicted_diseases = encoder.inverse_transform(y_pred)

# Print the predicted diseases
print("Predicted Diseases: ", predicted_diseases)

# Accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Voting Classifier (Hard Voting) Accuracy: {accuracy}")



[ 0  1  2  7  3  1  9  9  1  1 10  1  4  3  7  9  3  6  8  0  0  2  3  3
  5  7  0  2  6  5  4  1  0  6  5  4  0  9  5  9  0  5  9  5  1 10  0  0
  3  1  8 11  2  3  2  9  8  9  1  0  0  5  9  0  7  8  9  9  7 10  1  7
  5  0  0  0  0  2  0  7  6 10  5 11  3  1  0  5 10  1  2  1 10  9  3  6
  1  9  0 10  4  9  1  3  3  3  2 11 11  8  5  7  7  0  3  3  7  9  6  2
  0  4  2  3  1  2  3  4  3 10  3  8  9  0  4  2 11  0  9  5  9  2  6  2
  8  5  4  3  9  8  1  7  1  1  2  1  3  6  4  7  8  5  7 10  0 10  0  5
  2  0  9  5  1  9  9  4  0  9  6  5  0  2  3  9 10  1  9  5  4  0  2 11]
Predicted Diseases:  ['ADHD' 'ASD' 'Loneliness' 'anexiety' 'MDD' 'ASD' 'eating disorder'
 'eating disorder' 'ASD' 'ASD' 'psychotic deprission' 'ASD' 'OCD' 'MDD'
 'anexiety' 'eating disorder' 'MDD' 'PTSD' 'bipolar' 'ADHD' 'ADHD'
 'Loneliness' 'MDD' 'MDD' 'PDD' 'anexiety' 'ADHD' 'Loneliness' 'PTSD'
 'PDD' 'OCD' 'ASD' 'ADHD' 'PTSD' 'PDD' 'OCD' 'ADHD' 'eating disorder'
 'PDD' 'eating disorder' 'ADHD' 'PDD' 'eating d

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
!pip install streamlit -q
!pip install pyngrok

In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.199.149.142


In [ ]:


%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


# Load the model
model = joblib.load('model.pkl')

# Define features and their levels
levels = {
    "Level 1": [
        "ag+1:629e", "feeling.nervous", "trouble.in.concentration",
        "having.trouble.in.sleeping", "social.media.addiction",
        "having.nightmares", "change.in.eating", "feeling.tired"
    ],
    "Level 2": [
        "sweating", "breathing.rapidly", "anger", "close.friend",
        "introvert", "feeling.negative", "avoids.people.or.activities",
        "blamming.yourself"
    ],
    "Level 3": [
        "hallucinations", "panic", "hopelessness",
        "suicidal.thought", "popping.up.stressful.memory"
    ]
}

st.title("Mental Health Assessment Chatbot")

# Name and age input


# Store responses
responses = {}  # Use age input for "ag+1:629e"

# Loop through levels and features within them
for level, features in levels.items():
    with st.expander(level):
        st.subheader(level)  # Display level number prominently

        for feature in features:
            if feature == "ag+1:629e":  # Skip asking age again
              name = st.text_input("What is your name?")
              age = st.number_input("How old are you?", min_value=0, max_value=120, step=1)
              responses["ag+1:629e"]=age
              continue
            question = f"Do you experience {feature.replace('.', ' ')}?"
            user_response = st.selectbox(
                question,
                options=["Select an option", "Yes", "No"],
                key=feature
            )
            # Store response only if not placeholder

            responses[feature] = 1 if user_response == "Yes" else (0 if user_response == "No" else None)

if st.button("Submit"):
    if None in responses.values():
        st.warning("Please answer all the questions before submitting.")
    else:
        st.write(f"Thank you, {name}.")

        # Prediction
        input_data = pd.DataFrame([responses])
        prediction = model.predict(input_data)
        loaded_mapping = pd.read_csv('label_encoder_mappings.csv')
        for i in responses.keys():
          if responses[i]==1:
            responses[i]="yes"
          elif  responses[i]==0 :
            responses[i]="no"

        # Recreate the LabelEncoder
        encoder = LabelEncoder()
        encoder.classes_ = loaded_mapping['label'].values
        st.write(f"your responses are :{responses}")
        st.write(f"The model predicts: {encoder.inverse_transform([prediction[0]])}")


Overwriting app.py


['ADHD' 'ASD' 'Loneliness' 'MDD' 'OCD' 'PDD' 'PTSD' 'anexiety' 'bipolar'
 'eating disorder' 'psychotic deprission' 'sleeping disorder']
['MDD']


In [ ]:
!npm install localtunnel



up to date, audited 23 packages in 480ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

your url is: https://fruity-peas-say.loca.lt
^C
